In [8]:
import pandas as pd
import numpy as np

import numba

import bebi103

import altair as alt
import altair_catplot as altcat

import bokeh.io
import bokeh.plotting
bokeh.io.output_notebook()

color_palette=['#4e79a7', '#f28e2b', '#e15759', '#76b7b2', '#59a14f', '#edc948', '#b07aa1', '#ff9da7', '#9c755f', '#bab0ac']

Loading BokehJS ...

In [196]:
df = pd.read_csv('../data/hw_4.2_caulobacter_growth_image_processing_results.csv')

df.head()

,time (min),area (sq um),growth_event,bacterium
0,1.0,1.300624,0,1
1,2.0,1.314144,0,1
2,3.0,1.295216,0,1
3,4.0,1.314144,0,1
4,5.0,1.341184,0,1


Since the length of division is different for all growth events we should add another column that restarts the time count from 0 every time there's a division.

In [197]:
time = []
j = 0
for i in df['growth_event'].diff():
    if i == 0:
        j += 1
        time.append(j)
    else:
        j = 0
        time.append(j)

df['t'] = time

In [198]:
df.head()

,time (min),area (sq um),growth_event,bacterium,t
0,1.0,1.300624,0,1,0
1,2.0,1.314144,0,1,1
2,3.0,1.295216,0,1,2
3,4.0,1.314144,0,1,3
4,5.0,1.341184,0,1,4


Let's start with a subset of the data. We will look at bacterium 1.

In [199]:
df_bacterium1 = df.loc[df['bacterium'] == 1]

In [200]:
p = bokeh.plotting.figure(plot_width=650,
                          plot_height=250,
                          x_axis_label='time (min)',
                          y_axis_label='cell area (sq µm)')

# Specify the glyphs
colors = ['#1f78b4', '#a6cee3']
for i, g in df_bacterium1.groupby('growth_event'):
    p.circle(g['time (min)'], g['area (sq um)'], size=3, color=colors[i%2])

bokeh.io.show(p)

#### No hierarchy

Let's just model a single growth event for now with no hierarchy using the exponential model.

In [201]:
def data_prior_pred(t):
    '''
    Samples parameter values according to the prior and generates
    data y at the values given in t.
    '''
    # Sample parameter values according to priors
    a = np.random.normal(1.2, 0.4)
    k = np.random.normal(0.01, 0.003)
    sigma = np.abs(np.random.normal(0, 0.1))
    
    # Generate random data according to the likelihood
    return np.random.normal(a * np.exp(k * t), sigma)

In [202]:
p = bokeh.plotting.figure(height=300, width=450,
                          x_axis_label='time',
                          y_axis_label='area')

t = df_bacterium1.loc[df_bacterium1['growth_event'] == 0, 't'].values

# Plot simulated data
for i in range(100):
    p.circle(t, data_prior_pred(t), size=3, alpha=0.1)

# Plot original data
p.circle(t, df_bacterium1.loc[df_bacterium1['growth_event'] == 0, 'area (sq um)'].values, 
         color='black', size=4)
bokeh.io.show(p)

In [203]:
p = bokeh.plotting.figure(height=300, width=450,
                          x_axis_label='time',
                          y_axis_label='area')

t = df_bacterium1.loc[df_bacterium1['growth_event'] == 3, 't'].values

# Plot simulated data
for i in range(100):
    p.circle(t, data_prior_pred(t), size=3, alpha=0.1)

# Plot original data
p.circle(t, df_bacterium1.loc[df_bacterium1['growth_event'] == 3, 'area (sq um)'].values, 
         color='black', size=4)
bokeh.io.show(p)

Now let's try the linear model.

In [204]:
def data_prior_pred_linear(t):
    '''
    Samples parameter values according to the prior and generates
    data y at the values given in t.
    '''
    # Sample parameter values according to priors
    a = np.random.normal(1.2, 0.4)
    b = np.random.normal(0.01, 0.003)
    sigma = np.abs(np.random.normal(0, 0.1))
    
    # Generate random data according to the likelihood
    return np.random.normal(a + b * t, sigma)

In [205]:
p = bokeh.plotting.figure(height=300, width=450,
                          x_axis_label='time',
                          y_axis_label='area')

t = df_bacterium1.loc[df_bacterium1['growth_event'] == 0, 't'].values

# Plot simulated data
for i in range(100):
    p.circle(t, data_prior_pred_linear(t), size=3, alpha=0.1)

# Plot original data
p.circle(t, df_bacterium1.loc[df_bacterium1['growth_event'] == 0, 'area (sq um)'].values, 
         color='black', size=4)
bokeh.io.show(p)

#### One level hierarchical model

Let's try a one level hierarchical model for bacterium 1.

In [20]:
model_code_pri_pred = """
data {
  // Number of data points for each experiment
  int N;
  // Number of entries in each level of the hierarchy
  int J_1;
  // Input of parameters of the priors
  real hyper_a0_mu;
  real hyper_a0_sigma;
  
  real hyper_k0_mu;
  real hyper_k0_sigma;
  
  real hyper_a0_tau;
  real hyper_k0_tau;
}


generated quantities {
  // Total number of data points 
  real area[N * J_1];
  
  // Priors
  real a0 = normal_rng(hyper_a0_mu, hyper_a0_sigma);
  real k0 = normal_rng(hyper_k0_mu, hyper_k0_sigma);
  
  real a0_sigma = fabs(normal_rng(0, hyper_a0_sigma));
  real k0_sigma = fabs(normal_rng(0, hyper_k0_sigma));
  
  real a0_tau = fabs(normal_rng(0, hyper_a0_tau));
  real k0_tau = fabs(normal_rng(0, hyper_k0_tau));
  
  // Second layer
  real a_1[J_1]; 
  real k_1[J_1];
  
  for (i in 1:J_1) {
    a_1[i] = normal_rng(a0, a0_tau);
    k_1[i] = normal_rng(k0, k0_tau);
    for (j in 1:N) {
      area[(i - 1) * N + j] = normal_rng(a_1[i] * np.exp(k_1[i] * t), a0_sigma);
  }
  
  }
"""

In [21]:
# Compile
sm_gen = bebi103.stan.StanModel(model_code=model_code_pri_pred)

ValueError: Failed to parse Stan model 'anon_model_0ce0650fddca8bbfc0babc5f2673c893'. Error message:
SYNTAX ERROR, MESSAGE(S) FROM PARSER:

variable "t" does not exist.
  error in 'unknown file name' at line 41, column 68
  -------------------------------------------------
    39:     k_1[i] = normal_rng(k0, k0_tau);
    40:     for (j in 1:N) {
    41:       area[(i - 1) * N + j] = normal_rng(a_1[i] * np.exp(k_1[i] * t), a0_sigma);
                                                                           ^
    42:   }
  -------------------------------------------------



In [ ]:
# Number of iteration
N_iter = 1000

# input data from df
data = dict(N=100,
            J_1=3,
            hyper_a0_mu=1.4,
            hyper_a0_sigma=0.3,
            hyper_k0_mu=0.01,
            hyper_k0_sigma=0.002,
            hyper_a0_tau=0.1,
            hyper_k0_tau=0.001)

# Sample
df_pred = sm_gen.sampling(data=data,
                     algorithm='Fixed_param',
                     warmup=0,
                     chains=1,
                     iter=N_iter)

df_samples_a = bebi103.stan.extract_array(df_pred, name='a')
df_samples_k = bebi103.stan.extract_array(df_pred, name='k')

p_a = bebi103.viz.predictive_ecdf(df_pred, 
                                'a', 
                                x_axis_label='a')
p_k = bebi103.viz.predictive_ecdf(df_pred, 
                                'k', 
                                x_axis_label='k')
# Plot
bokeh.io.show(bokeh.layouts.gridplot([p_a, p_k], ncols=2))

Check for negative values:

In [ ]:
len(df_samples_a[df_samples_a['a'] < 0]) / len(df_samples_a)

In [ ]:
len(df_samples_k[df_samples_k['k'] < 0]) / len(df_samples_k)

In [ ]:
len(df_samples_a)

Prior predictive checks

In [ ]:
a = df_samples_a['a'].values
k = df_samples_k['k'].values

p = bokeh.plotting.figure(height=300, width=450,
                          x_axis_label='time',
                          y_axis_label='area')

t = df_bacterium1.loc[df_bacterium1['growth_event'] == 0, 't'].values

# Plot simulated data
for i in range(0, len(df_samples_a), 1000):
    p.circle(t, a[i] * np.exp(k[i] * t), size=3, alpha=0.1)

# Plot original data
p.circle(t, df_bacterium1.loc[df_bacterium1['growth_event'] == 0, 'area (sq um)'].values, 
         color='black', size=4)
bokeh.io.show(p)

Trying the same priors/samples for the linear model.

In [ ]:
a = df_samples_a['a'].values
k = df_samples_k['k'].values

p = bokeh.plotting.figure(height=300, width=450,
                          x_axis_label='time',
                          y_axis_label='area')

t = df_bacterium1.loc[df_bacterium1['growth_event'] == 0, 't'].values

# Plot simulated data
for i in range(0, len(df_samples_a), 1000):
    p.circle(t, a[i] + k[i] * t, size=3, alpha=0.1)

# Plot original data
p.circle(t, df_bacterium1.loc[df_bacterium1['growth_event'] == 0, 'area (sq um)'].values, 
         color='black', size=4)
bokeh.io.show(p)

prior predictive check trial 2

In [ ]:
model_code_pri_pred_2 = """
data {
  // Number of data points for each experiment
  int N;
  // Number of entries in each level of the hierarchy
  int J_1;
  // Time
  vector[N] t;
  
  // Input of parameters of the priors
  real hyper_a0_mu;
  real hyper_a0_sigma;
  
  real hyper_k0_mu;
  real hyper_k0_sigma;
  
  real hyper_a0_tau;
  real hyper_k0_tau;
  
  real sigma;
}


generated quantities {
  // Total number of data points 
  real a[N * J_1];
  real k[N * J_1];
  vector[N] area;
  
  // Priors
  real a0 = normal_rng(hyper_a0_mu, hyper_a0_sigma);
  real k0 = normal_rng(hyper_k0_mu, hyper_k0_sigma);
  
  real a0_sigma = fabs(normal_rng(0, hyper_a0_sigma));
  real k0_sigma = fabs(normal_rng(0, hyper_k0_sigma));
  
  real a0_tau = fabs(normal_rng(0, hyper_a0_tau));
  real k0_tau = fabs(normal_rng(0, hyper_k0_tau));
  
  // Second layer
  real a_1[J_1]; 
  real k_1[J_1];
  
  for (i in 1:J_1) {
    a_1[i] = normal_rng(a0, a0_tau);
    k_1[i] = normal_rng(k0, k0_tau);
    for (j in 1:N) {
      a[(i - 1) * N + j] = normal_rng(a_1[i], a0_sigma);
      k[(i - 1) * N + j] = normal_rng(k_1[i], k0_sigma);
    }
  }
  
  for (i in 1:N) {
    area[i] = random_rng(a[i] * exp(k[i] * t), sigma);
  }

  }
"""

# Compile
sm_gen = bebi103.stan.StanModel(model_code=model_code_pri_pred_2)

In [24]:
df_sub1 = df_bacterium1.loc[df_bacterium1['growth_event'] == 0]
df_sub2 = df_bacterium1.loc[df_bacterium1['growth_event'] == 1]
df_sub = pd.concat([df_sub1, df_sub2])

df_sub.head()

,time (min),area (sq um),growth_event,bacterium,t
0,1.0,1.300624,0,1,0
1,2.0,1.314144,0,1,1
2,3.0,1.295216,0,1,2
3,4.0,1.314144,0,1,3
4,5.0,1.341184,0,1,4


In [ ]:
# Number of iteration
N_iter = 1000

# input data from df
data = dict(N=len(df_sub),
            J_1=len(df_sub['growth_event'].unique()),
            t=df_sub['t'].values,
            hyper_a0_mu=1.4,
            hyper_a0_sigma=0.3,
            hyper_k0_mu=0.01,
            hyper_k0_sigma=0.002,
            hyper_a0_tau=0.1,
            hyper_k0_tau=0.001,
            sigma=0.1)

# Sample
df_pred = sm_gen.sampling(data=data,
                     algorithm='Fixed_param',
                     warmup=0,
                     chains=1,
                     iter=N_iter)

df_samples = bebi103.stan.extract_array(df_pred, name='area')

p = bebi103.viz.predictive_ecdf(df_pred, 
                                'area', 
                                x_axis_label='area')
# Plot
bokeh.io.show(p)

Noncentered model code:

In [22]:
model_code_linear_noncentered = """
data {
  // Total number of data points
  int N;
  
  // Number of entries in each level of the hierarchy
  int J_1;

  //Index arrays to keep track of hierarchical structure
  int index_1[N];
  
  // The measurements
  real area[N];
  
  // Time
  vector[N] t;
}

parameters {
  // Hyperparameters level 0
  real a;
  real k;
  real<lower=0> sigma;

  // How hyperparameters vary
  real<lower=0> tau_a;
  real<lower=0> tau_k;

  // Hyperparameters level 1
  vector[J_1] a_1_tilde;
  vector[J_1] k_1_tilde;
}

transformed parameters {
  // Transformations for noncentered
  vector[J_1] a_1 = a + tau_a * a_1_tilde;
  vector[J_1] k_1 = k + tau_k * k_1_tilde;
  vector[N] area_temp;
  
  for (i in 1:N) {
    area_temp[i] = a_1[index_1[i]] + k_1[index_1[i]] * t[i];
  }
}

model {
  a ~ normal(1.4, 0.3);
  k ~ normal(0.01, 0.002);
  sigma ~ normal(0, 0.1);
  tau_a ~ normal(0, 0.1);
  tau_k ~ normal(0, 0.001);

  a_1_tilde ~ normal(0, 1);
  k_1_tilde ~ normal(0, 1);

  area ~ normal(area_temp, sigma);
}

generated quantities {
  vector[N] area_ppc;
  
  for (i in 1:N) {
    area_ppc[i] = normal_rng(area_temp[i], sigma);
  }
}
"""

In [23]:
sm_noncentered = bebi103.stan.StanModel(model_code=model_code_linear_noncentered)

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_b7ee63f8ff19e32a900f7189db7c3844 NOW.
/Users/madelynw/anaconda3/lib/python3.7/site-packages/Cython/Compiler/Main.py:367: FutureWarning: Cython directive 'language_level' not set, using 2 for now (Py2). This will change in a later release! File: /var/folders/cn/x4zmqr494ln7sfp5z2kytzqw0000gp/T/tmpjskyce3n/stanfit4anon_model_b7ee63f8ff19e32a900f7189db7c3844_4972363008298383079.pyx
  tree = Parsing.p_module(s, pxd, full_module_name)


In [25]:
# Rename for convenience
df_bacterium1 = df_bacterium1.rename(columns={'area (sq um)': 'area'})

df_bacterium1.head()

,time (min),area,growth_event,bacterium,t
0,1.0,1.300624,0,1,0
1,2.0,1.314144,0,1,1
2,3.0,1.295216,0,1,2
3,4.0,1.314144,0,1,3
4,5.0,1.341184,0,1,4


In [104]:
df_sub1 = df_bacterium1.loc[df_bacterium1['growth_event'] == 1]
df_sub2 = df_bacterium1.loc[df_bacterium1['growth_event'] == 2]
df_sub = pd.concat([df_sub1, df_sub2])

df_sub.head()

,time (min),area,growth_event,bacterium,t
98,99.0,1.403376,1,1,0
99,100.0,1.400672,1,1,1
100,101.0,1.373632,1,1,2
101,102.0,1.406080,1,1,3
102,103.0,1.362816,1,1,4


In [105]:
data, df_part = bebi103.stan.df_to_datadict_hier(df_sub,
                                           level_cols=['growth_event'],
                                           data_cols=['area', 't'])

# Take a look
len(data['index_1'])

199

In [106]:
data

{'N': 199,
 'J_1': 2,
 'index_1': array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2]),
 'area': array([1.403376, 2.057744, 2.028   , 2.04152 , 2.022592, 1.998256,
        1.98744 , 1.979328, 1.9604  , 1.963104, 2.071264, 1.941472,
        1.900912, 1.890096, 1.881984, 1.873872, 1.857648, 1.854944,
        1.849536, 1.833312, 1.81168 , 1.91984 , 2.071264, 2.117232,
        

In [107]:
data = dict(N=len(df_sub),
            J_1=2,
            index_1=df_sub['growth_event'].values,
            area=df_sub['area'].values,
            t=df_sub['t'].values)

In [108]:
data

{'N': 199,
 'J_1': 2,
 'index_1': array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2]),
 'area': array([1.403376, 1.400672, 1.373632, 1.40608 , 1.362816, 1.387152,
        1.389856, 1.395264, 1.408784, 1.416896, 1.427712, 1.44664 ,
        1.44664 , 1.454752, 1.454752, 1.47368 , 1.470976, 1.503424,
        1.516944, 1.503424, 1.522352, 1.533168, 1.5548  , 1.562912,
        

In [153]:
# Sample
samples_linear = sm_noncentered.sampling(data=data, 
                                         seed=2389412, 
                                         control=dict(adapt_delta=0.99, max_treedepth=11))

# Convert to data frame for easy use later
df_linear = bebi103.stan.to_dataframe(samples_linear)

bebi103.stan.check_all_diagnostics(samples_linear)

n_eff / iter looks reasonable for all parameters.
Rhat looks reasonable for all parameters.
0 of 4000 (0.0%) iterations ended with a divergence.
0 of 4000 (0.0%) iterations saturated the maximum tree depth of 11.
E-BFMI indicated no pathological behavior.


0

In [110]:
bokeh.io.show(bebi103.viz.trace_plot(samples_linear, pars=['a', 'k'], line_width=2))

In [111]:
df_linear_ppc = bebi103.stan.extract_array(samples_linear, name='k_1')

In [112]:
bokeh.io.show(bebi103.viz.corner(samples_linear, pars=['a', 'k', 'sigma']))

In [149]:
def hw92_predictive(df, x, y=None, namex='index_1', name='F_ppc', perc=[80, 60, 40, 20], 
                    x_axis_label=None, y_axis_label=None, title=None, plot_width=350, plot_height=225, 
                    color='blue', data_color=color_palette[1], diff=False, p=None):
    '''Mimic of predictive ECDF
    df - MCMC sampling data frame
    x - input variable
    y - data
    namex - name of the input varible in the data frame
    name - name of the predictive results in the data frame
    perc - list, default [80, 60, 40, 20]
            Percentiles for making colored envelopes for confidence
            intervals for the predictive ECDFs. Maximally four can be 
            specified.'''
    
    if color not in ['green', 'blue', 'red', 'gray',
                     'purple', 'orange', 'betancourt']:
        raise RuntimeError("Only allowed colors are 'green', 'blue', 'red', 'gray', 'purple', 'orange'")
    
    colors = {'blue': ['#9ecae1','#6baed6','#4292c6','#2171b5','#084594'],
              'green': ['#a1d99b','#74c476','#41ab5d','#238b45','#005a32'],
              'red': ['#fc9272','#fb6a4a','#ef3b2c','#cb181d','#99000d'],
              'orange': ['#fdae6b','#fd8d3c','#f16913','#d94801','#8c2d04'],
              'purple': ['#bcbddc','#9e9ac8','#807dba','#6a51a3','#4a1486'],
              'gray': ['#bdbdbd','#969696','#737373','#525252','#252525'],
              'betancourt': ['#DCBCBC', '#C79999', '#B97C7C',
                             '#A25050', '#8F2727', '#7C0000']}
    if p is None:
        p = bokeh.plotting.figure(plot_width=plot_width,
                                  plot_height=plot_height,
                                  x_axis_label=x_axis_label,
                                  y_axis_label=y_axis_label,
                                  title=title)
    
    if diff:
        x = x[1:]
        if y is not None:
            y = np.diff(y)
        Nb = len(x)
        y_ppc = np.empty((len(perc) * 2 + 1, Nb))
        for i in range(Nb):
            temp = df.loc[df[namex]== i+2, name].values - df.loc[df[namex]== i+1, name].values
            y_ppc[-1, i] = np.median(temp)
            for j in range(len(perc)):
                y_ppc[j * 2, i] = np.percentile(temp, 50 - perc[j] / 2)
                y_ppc[j * 2 + 1, i] = np.percentile(temp, 50 + perc[j] / 2)
    else:                
        Nb = len(x)
        y_ppc = np.empty((len(perc) * 2 + 1, Nb))
        for i in range(Nb):
            temp = df.loc[df[namex]== i+1, name].values
            y_ppc[-1, i] = np.median(temp)
            for j in range(len(perc)):
                y_ppc[j * 2, i] = np.percentile(temp, 50 - perc[j] / 2)
                y_ppc[j * 2 + 1, i] = np.percentile(temp, 50 + perc[j] / 2)
    
    for j in range(len(perc)):
        bebi103.viz.fill_between(x, y_ppc[j * 2, :],
                     x, y_ppc[j * 2 + 1,:],
                     p=p,
                     show_line=False,
                     fill_color=colors[color][j])
        
    p.circle(x, y_ppc[-1, :],
           size=4,
           color=colors[color][-1])
    
    if y is not None:
        p.circle(x, y, size=4, color='orange')
    
    return p

In [154]:
time = df_sub['time (min)'].values
val = df_sub['area'].values
df_linear_ppc = bebi103.stan.extract_array(samples_linear, name='area_ppc')

In [160]:
# df_linear_ppc['index_1'].unique()

In [161]:
# len(df_sub.loc[df_sub['growth_event'] == 1,'time (min)'].values)

In [155]:
# time = df_sub.loc[df_sub['growth_event'] == 1,'time (min)'].values
# val = df_sub.loc[df_sub['growth_event'] == 1,'area'].values

# p1 = hw92_predictive(df_linear_ppc, time, val, perc=[99, 75, 50, 25], name='area_ppc', plot_width=500, plot_height=400)

# time = df_sub.loc[df_sub['growth_event'] == 2,'time (min)'].values
# val = df_sub.loc[df_sub['growth_event'] == 2,'area'].values

p1 = hw92_predictive(df_linear_ppc, time, val, perc=[99, 75, 50, 25], name='area_ppc', plot_width=500, plot_height=400)

bokeh.io.show(p1)

In [159]:
# df_linear_ppc[df_linear_ppc['chain_idx'] == 1]

In [ ]:
# Marginalized distributions of each parameter 
plots = [bebi103.viz.ecdf(df_linear[param], x_axis_label=param, plot_height=200, plot_width=250) 
                 for param in ['a', 'k']]
bokeh.io.show(bokeh.layouts.gridplot(plots, ncols=3))

Noncentered exponential model

In [214]:
model_code_exp_noncentered = """
data {
  // Total number of data points
  int N;
  
  // Number of entries in each level of the hierarchy
  int J_1;

  //Index arrays to keep track of hierarchical structure
  int index_1[N];
  
  // The measurements
  real area[N];
  
  // Time
  vector[N] t;
}

parameters {
  // Hyperparameters level 0
  real a;
  real k;
  real<lower=0> sigma;

  // How hyperparameters vary
  real<lower=0> tau_a;
  real<lower=0> tau_k;

  // Hyperparameters level 1
  vector[J_1] a_1_tilde;
  vector[J_1] k_1_tilde;
}

transformed parameters {
  // Transformations for noncentered
  vector[J_1] a_1 = a + tau_a * a_1_tilde;
  vector[J_1] k_1 = k + tau_k * k_1_tilde;
  vector[N] area_temp;
  
  for (i in 1:N) {
    area_temp[i] = a_1[index_1[i]] * exp(k_1[index_1[i]] * t[i] / 100);
  }
}

model {
  a ~ normal(1.4, 0.3);
  k ~ normal(1, 0.2);
  sigma ~ normal(0, 0.1);
  tau_a ~ normal(0, 0.1);
  tau_k ~ normal(0, 0.1);

  a_1_tilde ~ normal(0, 1);
  k_1_tilde ~ normal(0, 1);

  area ~ normal(area_temp, sigma);
}

generated quantities {
  vector[N] area_ppc;
  
  for (i in 1:N) {
    area_ppc[i] = normal_rng(area_temp[i], sigma);
  }
}
"""

see t9b for log_likelihood -- normal_lpdf

In [215]:
sm_exp = bebi103.stan.StanModel(model_code=model_code_exp_noncentered)

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_9a0ec04e93efb3947fb9d16ff22eb0b7 NOW.
/Users/madelynw/anaconda3/lib/python3.7/site-packages/Cython/Compiler/Main.py:367: FutureWarning: Cython directive 'language_level' not set, using 2 for now (Py2). This will change in a later release! File: /var/folders/cn/x4zmqr494ln7sfp5z2kytzqw0000gp/T/tmpftq6__2a/stanfit4anon_model_9a0ec04e93efb3947fb9d16ff22eb0b7_6083595218447090237.pyx
  tree = Parsing.p_module(s, pxd, full_module_name)


In [217]:
# [dict(a=1.3, k=0.01)]*4

In [218]:
# [{('a', 'k'): (s, t)} for s,t in np.random.normal(1.3, 0.2, size=4), np.random.normal(0.01, 0.002, size=4)]

In [216]:
# Sample
samples_exp = sm_exp.sampling(data=data, 
                              seed=2389412, 
                              control=dict(adapt_delta=0.99, max_treedepth=11))

# Convert to data frame for easy use later
df_exp = bebi103.stan.to_dataframe(samples_exp)

bebi103.stan.check_all_diagnostics(samples_exp)

n_eff / iter looks reasonable for all parameters.
Rhat looks reasonable for all parameters.
0 of 4000 (0.0%) iterations ended with a divergence.
2620 of 4000 (65.5%) iterations saturated the maximum tree depth of 10.
  Try running again with max_treedepth set to a larger value to avoid saturation.
E-BFMI indicated no pathological behavior.


8

In [219]:
bokeh.io.show(bebi103.viz.trace_plot(samples_exp, pars=['a', 'k'], line_width=2))

In [220]:
bokeh.io.show(bebi103.viz.corner(samples_exp, pars=['a', 'k']))

In [221]:
time = df_sub['time (min)'].values
val = df_sub['area'].values
df_exp_ppc = bebi103.stan.extract_array(samples_exp, name='area_ppc')

In [222]:
p1 = hw92_predictive(df_exp_ppc, time, val, perc=[99, 75, 50, 25], name='area_ppc', plot_width=500, plot_height=400)

bokeh.io.show(p1)

#### Two levels

In [224]:
model_code_linear_2 = """
data {
  // Total number of data points
  int N;
  
  // Number of entries in each level of the hierarchy
  int J_1;
  int J_2;
  
  //Index arrays to keep track of hierarchical structure
  int index_1[J_2];
  int index_2[N];
  
  // The measurements
  real area[N];
  
  // Time
  vector[N] t;
}

parameters {
  // Hyperparameters level 0
  real a;
  real k;
  real<lower=0> sigma;

  // How hyperparameters vary
  real<lower=0> tau_a;
  real<lower=0> tau_k;

  // Hyperparameters level 1
  vector[J_1] a_1_tilde;
  vector[J_1] k_1_tilde;
  
  // Hyperparameters level 2
  vector[J_2] a_2_tilde;
  vector[J_2] k_2_tilde;
}

transformed parameters {
  // Transformations for noncentered
  vector[J_1] a_1 = a + tau_a * a_1_tilde;
  vector[J_1] k_1 = k + tau_k * k_1_tilde;
  
  vector[J_2] a_2 = a_1[index_1] + tau_a * a_2_tilde;
  vector[J_2] k_2 = k_1[index_1] + tau_k * k_2_tilde;
  
  vector[N] area_temp;
  
  for (i in 1:N) {
    area_temp[i] = a_2[index_2[i]] + k_2[index_2[i]] * t[i];
  }
}

model {
  a ~ normal(1.4, 0.3);
  k ~ normal(0.01, 0.002);
  sigma ~ normal(0, 0.1);
  tau_a ~ normal(0, 0.1);
  tau_k ~ normal(0, 0.001);

  a_1_tilde ~ normal(0, 1);
  k_1_tilde ~ normal(0, 1);
  
  a_2_tilde ~ normal(0, 1);
  k_2_tilde ~ normal(0, 1);

  area ~ normal(area_temp, sigma);
}

generated quantities {
  vector[N] area_ppc;
  
  for (i in 1:N) {
    area_ppc[i] = normal_rng(area_temp[i], sigma);
  }
}
"""

In [225]:
sm_linear_2 = bebi103.stan.StanModel(model_code=model_code_linear_2)

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_1ea7e02dd299844a351cb1ab33e1cf83 NOW.
/Users/madelynw/anaconda3/lib/python3.7/site-packages/Cython/Compiler/Main.py:367: FutureWarning: Cython directive 'language_level' not set, using 2 for now (Py2). This will change in a later release! File: /var/folders/cn/x4zmqr494ln7sfp5z2kytzqw0000gp/T/tmpyl2uoftv/stanfit4anon_model_1ea7e02dd299844a351cb1ab33e1cf83_696028066886628464.pyx
  tree = Parsing.p_module(s, pxd, full_module_name)


In [265]:
df.loc[df['bacterium'] == 2]

,time (min),area (sq um),growth_event,bacterium,t
1888,1.0,1.460160,0,2,0
1889,2.0,1.443936,0,2,1
1890,3.0,1.443936,0,2,2
1891,4.0,1.473680,0,2,3
1892,5.0,1.476384,0,2,4
1893,6.0,1.506128,0,2,5
1894,7.0,1.481792,0,2,6
1895,8.0,1.503424,0,2,7
1896,9.0,1.516944,0,2,8
1897,10.0,1.519648,0,2,9


In [234]:
df_sub1 = df.loc[(df['growth_event'] == 1) & (df['bacterium'] == 1)]
df_sub2 = df.loc[(df['growth_event'] == 2) & (df['bacterium'] == 1)]
df_sub3 = df.loc[(df['growth_event'] == 3) & (df['bacterium'] == 2)]
df_sub = pd.concat([df_sub1, df_sub2])
df_sub = pd.concat([df_sub, df_sub3])

df_sub.head()

,time (min),area (sq um),growth_event,bacterium,t
98,99.0,1.403376,1,1,0
99,100.0,1.400672,1,1,1
100,101.0,1.373632,1,1,2
101,102.0,1.406080,1,1,3
102,103.0,1.362816,1,1,4


In [236]:
# Rename for convenience
df_sub = df_sub.rename(columns={'area (sq um)': 'area'})

In [246]:
data, df_part = bebi103.stan.df_to_datadict_hier(df_sub,
                                           level_cols=['bacterium', 'growth_event'],
                                           data_cols=['area', 't'])

# Take a look
data

{'N': 288,
 'J_1': 2,
 'J_2': 3,
 'index_1': array([1, 1, 2]),
 'index_2': array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
        3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
        3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
        3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 

In [259]:
data = dict(N=len(df_sub),
            J_1=2,
            J_2=3,
            index_1=np.array([1,1,2]),
            index_2=df_sub['growth_event'].values,
            area=df_sub['area'].values,
            t=df_sub['t'].values)

In [260]:
data

{'N': 288,
 'J_1': 2,
 'J_2': 3,
 'index_1': array([1, 1, 2]),
 'index_2': array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
        3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
        3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
        3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 

In [261]:
# Sample
samples_linear_2 = sm_linear_2.sampling(data=data, 
                              seed=2389412, 
                              control=dict(adapt_delta=0.99, max_treedepth=11))

# Convert to data frame for easy use later
df_linear_2 = bebi103.stan.to_dataframe(samples_linear_2)

bebi103.stan.check_all_diagnostics(samples_linear_2)

n_eff / iter looks reasonable for all parameters.
Rhat looks reasonable for all parameters.
0 of 4000 (0.0%) iterations ended with a divergence.
640 of 4000 (16.0%) iterations saturated the maximum tree depth of 11.
  Try running again with max_treedepth set to a larger value to avoid saturation.
E-BFMI indicated no pathological behavior.


8

In [262]:
bokeh.io.show(bebi103.viz.trace_plot(samples_linear_2, pars=['a', 'k'], line_width=2))

In [263]:
bokeh.io.show(bebi103.viz.corner(samples_linear_2, pars=['a', 'k']))

In [264]:
time = df_sub['time (min)'].values
val = df_sub['area'].values
df_lin2_ppc = bebi103.stan.extract_array(samples_linear_2, name='area_ppc')

p1 = hw92_predictive(df_lin2_ppc, time, val, perc=[99, 75, 50, 25], name='area_ppc', plot_width=500, plot_height=400)

bokeh.io.show(p1)

In [ ]:
%load_ext watermark

In [ ]:
%watermark -v -p numpy,scipy,bokeh,jupyterlab